In [1]:
%load_ext autoreload

In [10]:
%autoreload

import sys
import os
import json

# os.environ[ "LANGCHAIN_WANDB_TRACING" ] = "true"
# # wandb documentation to configure wandb using env variables
# # https://docs.wandb.ai/guides/track/advanced/environment-variables
# # here we are configuring the wandb project name
# os.environ[ "WANDB_PROJECT" ] = "deepily-dataframe-agent"

path = "/var/genie-in-the-box/src/lib"
if path not in sys.path:
    sys.path.append( path )
else:
    print( f"[{path}] already in sys.path" )

print( sys.path )

import lib.util as du

path = "/var/genie-in-the-box/src"
du.add_to_path( path )

import lib.util_pandas      as up
import lib.util_stopwatch   as sw
import lib.util_code_runner as ucr

from agent_calendaring import CalendaringAgent

import pandas as pd
import openai

debug = True
verbose = True


[/var/genie-in-the-box/src/lib] already in sys.path
['', '/Applications/PyCharm.app/Contents/plugins/python/helpers-pro/jupyter_debug', '/Applications/PyCharm.app/Contents/plugins/python/helpers/pydev', '/usr/local/lib/python3.10', '/usr/local/lib/python3.10/lib-dynload', '/usr/local/lib/python3.10/site-packages', '/usr/local/lib/python310.zip', '/var/genie-in-the-box/src', '/var/genie-in-the-box/src/lib', '/var/genie-in-the-box/src/notebooks']
Path [/var/genie-in-the-box/src] already in sys.path


In [9]:
GPT_4   = "gpt-4-0613"
GPT_3_5 = "gpt-3.5-turbo-0613"

def ask_chat_gpt_text( preamble, query, model=GPT_4, debug=False ):

    openai.api_key = os.getenv( "FALSE_POSITIVE_API_KEY" )
    
    timer = sw.Stopwatch( msg=f"Asking ChatGPT [{model}]...".format( model ) )
    
    response = openai.ChatCompletion.create(
        model=model,
        messages=[ 
            { "role": "system", "content": preamble },
            { "role": "user",   "content": query } 
        ],
        temperature=0,
        max_tokens=2400,
        top_p=1.0,
        frequency_penalty=0.0,
        presence_penalty=0.0
    )
    timer.print( use_millis=True )
    
    if debug: print( json.dumps( response, indent=4 ) )
    
    return response[ "choices" ][ 0 ][ "message" ][ "content" ].strip()

preamble = "You are chatGPT, a helpful chatbot"
question = "What's your name?"

ask_chat_gpt_text( preamble, question, model=GPT_4 )

Asking ChatGPT [gpt-4-0613]...
Asking ChatGPT [gpt-4-0613]... in 1,692 ms


"My name is ChatGPT. I'm an artificial intelligence designed to assist with information and conversation."

In [21]:
from solution_snapshot_mgr import SolutionSnapshotManager
from agent_refactoring     import RefactoringAgent

In [34]:
%autoreload

path_to_snapshots = du.get_project_root() + "/src/conf/long-term-memory/solutions/"
snapshot_mgr      = SolutionSnapshotManager( path_to_snapshots, debug=False )
exemplar_snapshot = snapshot_mgr.get_snapshots_by_question( "What concerts do I have this week?" )[ 0 ][ 1 ]
similar_snapshots = snapshot_mgr.get_snapshots_by_code_similarity( exemplar_snapshot, threshold=90.0 )

agent         = RefactoringAgent( similar_snapshots=similar_snapshots, path_to_solutions="/src/conf/long-term-memory/solutions", debug=True, verbose=True )
response_dict = agent.run_prompt()


------------------------------------------------------------------------------------------------------------------------
- Found [32] synonymous questions
------------------------------------------------------------------------------------------------------------------------

Synonymous question [when is juans birthday] for snapshot.question [when is juans birthday]
Synonymous question [when was juan born] for snapshot.question [when is juans birthday]
Synonymous question [what day is today] for snapshot.question [what day is today]
Synonymous question [whats todays date] for snapshot.question [what day is today]
Synonymous question [whats todays day and date] for snapshot.question [what day is today]
Synonymous question [what is the day and date for today] for snapshot.question [what day is today]
Synonymous question [whats the date] for snapshot.question [what day is today]
Synonymous question [say what day is today] for snapshot.question [what day is today]
Synonymous question [hey

In [38]:
agent.response_dict[ "gpt_function_signature" ]

"{'name': 'get_events_by_type_and_date', 'description': 'Gets events of a given type within a date range', 'parameters': {'type': 'object', 'properties': {'df': {'type': 'DataFrame', 'description': 'The DataFrame containing the events'}, 'event_type': {'type': 'string', 'description': 'The type of event'}, 'start_date': {'type': 'Timestamp', 'description': 'The start date of the range'}, 'end_date': {'type': 'Timestamp', 'description': 'The end date of the range'}}, 'required': ['df', 'event_type']}}"

In [41]:
%autoreload
agent.refactor_code( debug=False )

Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_0.py]... Done!
Writing file [/var/genie-in-the-box/src/lib/autogen/util_calendaring_0.json]... Done!
Writing file [/var/genie-in-the-box/io/lib/autogen/util_calendaring_0.py]... Done!

Generating embedding for [import lib.autogen.util_calendaring_0 as uc0 print(uc0.get_event...]...
Generating embedding for [import lib.autogen.util_calendaring_0 as uc0 print(uc0.get_event...]... Done! in 604 ms

solution_file value provided: [what-concerts-do-i-have-this-week-0.json]...
File path: /var/genie-in-the-box/src/conf/long-term-memory/solutions/what-concerts-do-i-have-this-week-0.json
Generating embedding for [import lib.autogen.util_calendaring_0 as uc0 print(uc0.get_event...]...
Generating embedding for [import lib.autogen.util_calendaring_0 as uc0 print(uc0.get_event...]... Done! in 505 ms

solution_file value provided: [what-birthdays-do-i-have-this-week-0.json]...
File path: /var/genie-in-the-box/src/conf/long-term-memor

In [31]:
%autoreload
agent.run_code()

last command, before [print(uc0.get_events_by_week_and_type(df, 'Concert'))]:
return_type [string]
ERROR: return_type [string] and last command [print(uc0.get_events_by_week_and_type(df, 'Concert'))] are incompatible!
last command,  after [print(uc0.get_events_by_week_and_type(df, 'Concert'))]:
import pandas as pd
import lib.util as du
import lib.util_pandas as dup

debug = True

df = pd.read_csv( du.get_project_root() + '/src/conf/long-term-memory/events.csv' )
df = dup.cast_to_datetime( df, debug=debug )
import lib.autogen.util_calendaring_0 as uc0
print(uc0.get_events_by_week_and_type(df, 'Concert'))

Executing /var/genie-in-the-box/io/code.py... Done!

------------------------------------------------------------------------------------------------------------------------
- Code output
------------------------------------------------------------------------------------------------------------------------

{"start_date":1695859200000,"end_date":1696377600000,"start_time":"00:00","end

[{'return_code': 0,
  'output': '{"start_date":1695859200000,"end_date":1696377600000,"start_time":"00:00","end_time":"23:59","event_type":"Concert","recurrent":false,"recurrence_interval":null,"priority_level":"low","name":"Leroy Ruiz","relationship":"father","description_who_what_where":"Concert of Leroy Ruiz at the city center"}\n{"start_date":1696118400000,"end_date":1696204800000,"start_time":"00:00","end_time":"23:59","event_type":"Concert","recurrent":false,"recurrence_interval":null,"priority_level":"highest","name":"Gregorio","relationship":"friend","description_who_what_where":"Concert of Gregorio at the city center"}\n{"start_date":1696118400000,"end_date":1696377600000,"start_time":"00:00","end_time":"23:59","event_type":"Concert","recurrent":false,"recurrence_interval":null,"priority_level":"highest","name":"John","relationship":"coworker","description_who_what_where":"Concert of John at the city center"}'},
 {'return_code': 0,
  'output': '{"start_date":1695772800000,"end